 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/0.%20Download%20and%20Preprocess%20Data.ipynb#Downloading-publication-data-from-Arxiv" data-toc-modified-id="Downloading-publication-data-from-Arxiv-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Downloading publication data from Arxiv</a></span></li><li><span><a href="http://localhost:8888/notebooks/0.%20Download%20and%20Preprocess%20Data.ipynb#Downloading-publication-data-from-DBLP" data-toc-modified-id="Downloading-publication-data-from-DBLP-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Downloading publication data from DBLP</a></span></li></ul></div>

This is a shell notebook that downloads all of the data from arxiv.org, DBLP, and csrankings.org and calls the Python scripts that do preprocessing. This notebook will take about an hour to run and download several gigabytes of data from the Web.

These scripts require `wget`, Python 2, and `nltk`.

The downloaded files are placed in a subdirectory of the current folder. First you might want to check the output of the below and make sure this is running in a directory that you are happy with.

In [1]:
echo $PWD

/Users/csutton/research/projects/biblio/arxiv/SuttonGongCSArxiv


# Downloading publication data from Arxiv

First we create the new directories that we need.

In [2]:
OUTDIR=$PWD/generated/arxiv
mkdir -p $OUTDIR
mkdir $OUTDIR/raw/
mkdir $OUTDIR/json/

The scripts that we want to run live in the directory `arxiv/`. They are hardcoded to download files to the directories that we created just above. 

This script downloads the metadata of the CS and STATS papers from Arxiv using the OAI2 interface as XML. The script makes multiple downloads to handle the paging requirements of OAI. These are all downloaded to `generated/arxiv/raw`. No other processing is performed at this stage.

In [3]:
python2 arxiv/download_arxiv.py

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177


Now, the next script combines all of the downloaded XML files into a single JSON file, so that they are easier to load later. This also removes some arxiv metadata, like the abstracts, that we don't need in our processing, and extracts the datestamp year into a separate json field, because we especially care about it. Finally, the script tokenizes the title and authors into a bag of words and places the resulting vectors into new fields in the JSON records; we'll use this for record linkage later.

In [5]:
python2 arxiv/xml2json_arxiv.py

Processing 177129 papers from arxiv
.................................................................................................................................................................................

Save the date we downloaded everything.

In [6]:
date > generated/arxiv/arxiv-date-downloaded.txt

# Downloading publication data from DBLP

First create the directories we need:

In [2]:
DBLPDIR=$PWD/generated/dblp
mkdir -p $DBLPDIR

The DBLP archive can be downloaded from a single file:

In [8]:
wget -O $DBLPDIR/dblp.orig.xml.gz http://dblp.uni-trier.de/xml/dblp.xml.gz

--2017-09-20 11:24:13--  http://dblp.uni-trier.de/xml/dblp.xml.gz
Resolving dblp.uni-trier.de... 136.199.55.186
Connecting to dblp.uni-trier.de|136.199.55.186|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406271748 (387M) [application/x-gzip]
Saving to: ‘/Users/csutton/research/projects/biblio/arxiv/SuttonGongCSArxiv/generated/dblp/dblp.xml.gz’

/Users/csutton/rese 100%[===================>] 387.45M  3.12MB/s    in 2m 3s   

2017-09-20 11:26:16 (3.16 MB/s) - ‘/Users/csutton/research/projects/biblio/arxiv/SuttonGongCSArxiv/generated/dblp/dblp.xml.gz’ saved [406271748/406271748]



This command replaces XML entities with the actual characters, so that we can just use Unicode with future processing.

In [4]:
sh dblp/fix-dblp.sh dblp $DBLPDIR/dblp.orig.xml.gz $DBLPDIR/dblp.xml.gz

Reading DTD from dblp


The scripts in the dblp/ directory are modified versions of the ones here, used to generated csrankings.org https://github.com/emeryberger/CSrankings/tree/gh-pages/util. What this script does is filter all of the publications in DBLP according to the list of venues and years that we are interested in. The lists of venues and years are specified in dblp/setup.py

In [14]:
time python2 dblp/regenerate-data.py

82427

real	16m39.121s
user	15m39.779s
sys	0m11.387s


Save the date we downloaded everything.

In [15]:
date > generated/dblp/dblp-date-downloaded.txt

Now proceed to notebook 1